# SAIL 2025 – Prep Notebook (Flows + Sensor Locations → Dashboard Files)

Deze notebook:
1. Laadt je **crowd flow** dataset (3-minuten resolutie) en **sensor-locaties**.
2. Maakt drie outputbestanden die direct bruikbaar zijn in je dashboard/notebooks:
   - `/mnt/data/crowd_timeseries.csv` – long-form timeseries met `timestamp, zone_id, density` (density = flow per 3 min als proxy).
   - `/mnt/data/zones.geojson` – simpele polygonen rond sensoren voor choropleth-kaart.
   - `/mnt/data/crowd_timeseries_features.csv` – zelfde als `crowd_timeseries.csv` maar inclusief lege kolommen voor AIS/KNMI/WBGT (zodat je later eenvoudig kunt joinen).
3. **Nodig om te draaien:** alleen jouw twee bestaande CSV-bestanden.

## 1) Bestanden (alleen deze twee zijn nu nodig)

In [15]:
from pathlib import Path

# <<< VERVANG ALLEEN ALS JE EIGEN PAD WILT GEBRUIKEN >>>
FLOW_CSV = Path("sensordata_SAIL2025.csv")
LOC_CSV  = Path("sensor-location.xlsx - Sheet1.csv")

# Output-bestanden (laat zo staan)
OUT_TIMESERIES = Path("output_notebook_chat/crowd_timeseries.csv")
OUT_GEOJSON    = Path("output_notebook_chat/zones.geojson")
OUT_FEATURES   = Path("output_notebook_chat/crowd_timeseries_features.csv")

# Timeseries is 3-min resolutie
STEP_MINUTES = 3
print(FLOW_CSV.exists(), LOC_CSV.exists())

True True


## 2) Imports

In [16]:
import pandas as pd
import numpy as np
import json
from datetime import timedelta

## 3) Preview data

In [17]:
# Lees een snelle peek om kolommen te zien
df_preview = pd.read_csv(FLOW_CSV, nrows=3)
loc_preview = pd.read_csv(LOC_CSV, nrows=3)
display(df_preview.head(3))
display(df_preview.columns.tolist())
display(loc_preview.head(3))
display(loc_preview.columns.tolist())

,timestamp,CMSA-GAKH-01_0,CMSA-GAKH-01_180,CMSA-GAWW-11_120,CMSA-GAWW-11_300,CMSA-GAWW-12_115,CMSA-GAWW-12_295,CMSA-GAWW-13_120,CMSA-GAWW-13_300,CMSA-GAWW-14_40,...,GVCV-13_10,GVCV-13_190,GVCV-14_90,GVCV-14_270,hour,minute,day,month,weekday,is_weekend
0,2025-08-20 00:00:00+02:00,15,4,29,33,44,28,42,37,11,...,41,40,0,0,0,0,20,8,2,0
1,2025-08-20 00:03:00+02:00,1,3,21,29,34,39,9,14,6,...,0,0,0,0,0,3,20,8,2,0
2,2025-08-20 00:06:00+02:00,5,4,35,22,29,34,33,42,14,...,0,0,0,0,0,6,20,8,2,0


['timestamp',
 'CMSA-GAKH-01_0',
 'CMSA-GAKH-01_180',
 'CMSA-GAWW-11_120',
 'CMSA-GAWW-11_300',
 'CMSA-GAWW-12_115',
 'CMSA-GAWW-12_295',
 'CMSA-GAWW-13_120',
 'CMSA-GAWW-13_300',
 'CMSA-GAWW-14_40',
 'CMSA-GAWW-14_220',
 'CMSA-GAWW-15_30',
 'CMSA-GAWW-15_210',
 'CMSA-GAWW-16_30',
 'CMSA-GAWW-16_210',
 'CMSA-GAWW-17_40',
 'CMSA-GAWW-17_220',
 'CMSA-GAWW-19_115',
 'CMSA-GAWW-19_295',
 'CMSA-GAWW-20_120',
 'CMSA-GAWW-20_300',
 'CMSA-GAWW-21_120',
 'CMSA-GAWW-21_300',
 'CMSA-GAWW-23_109',
 'CMSA-GAWW-23_289',
 'GACM-04_50',
 'GACM-04_230',
 'GASA-01-A1_135',
 'GASA-01-A1_315',
 'GASA-01-A2_135',
 'GASA-01-A2_315',
 'GASA-01-B_135',
 'GASA-01-B_315',
 'GASA-01-C_135',
 'GASA-01-C_315',
 'GASA-02-01_135',
 'GASA-02-01_315',
 'GASA-02-02_135',
 'GASA-02-02_315',
 'GASA-03_105',
 'GASA-03_285',
 'GASA-04_135',
 'GASA-04_315',
 'GASA-05-O_135',
 'GASA-05-O_315',
 'GASA-05-W_135',
 'GASA-05-W_315',
 'GASA-06_95',
 'GASA-06_275',
 'GASA-06-B_95',
 'GASA-06-B_275',
 'GVCV-01_40',
 'GVCV-01_220',


,Objectummer,Locatienaam,Lat/Long,Breedte,Effectieve breedte
0,CMSA-GAKH-01,Kalverstraat t.h.v. 1,"52.372634, 4.892071",8,"6,7"
1,CMSA-GAWW-11,Korte Niezel,"52.374616, 4.899830","3,8","3,4"
2,CMSA-GAWW-12,Oudekennissteeg,"52.373860, 4.898690",3,"2,6"


['Objectummer', 'Locatienaam', 'Lat/Long', 'Breedte', 'Effectieve\xa0 breedte']

## 4) Flow → long-form timeseries

- We detecteren alle sensorkolommen (alles behalve `timestamp`, helper-kolommen).
- Sensoren met richting-suffix (bijv. `_0`, `_180`) worden **samengevoegd per basenaam** (som over richtingen).
- We vormen `density = flow_3min` als eenvoudige proxy (later kun je dit kalibreren).

In [18]:
# 4.a Lees volledige flow CSV
df = pd.read_csv(FLOW_CSV)
df['timestamp'] = pd.to_datetime(df['timestamp'])

# Helper/metadata kolommen die we NIET willen optellen
helper_cols = {'hour','minute','day','month','weekday','is_weekend','timestamp'}
sensor_cols = [c for c in df.columns if c not in helper_cols]

# Functie om basenaam (zonder richting) te pakken: alles voor de laatste underscore
def base_sensor(col):
    return col.rsplit('_',1)[0] if '_' in col else col

base_map = {c: base_sensor(c) for c in sensor_cols}

# 4.b Som over richtingen per basenaam
df_summed = df[['timestamp']].copy()
for base in sorted(set(base_map.values())):
    cols = [c for c,b in base_map.items() if b == base]
    df_summed[base] = df[cols].sum(axis=1)

# 4.c Long-form
long_df = df_summed.melt(id_vars='timestamp', var_name='zone_id', value_name='flow_3min')
long_df['density'] = long_df['flow_3min'].astype(float)  # proxy
long_df = long_df[['timestamp','zone_id','density']].sort_values(['zone_id','timestamp']).reset_index(drop=True)

display(long_df.head())
print("Rows:", len(long_df), "Zones:", long_df['zone_id'].nunique(),
      "Start:", long_df['timestamp'].min(), "End:", long_df['timestamp'].max())

,timestamp,zone_id,density
0,2025-08-20 00:00:00+02:00,CMSA-GAKH-01,19.0
1,2025-08-20 00:03:00+02:00,CMSA-GAKH-01,4.0
2,2025-08-20 00:06:00+02:00,CMSA-GAKH-01,9.0
3,2025-08-20 00:09:00+02:00,CMSA-GAKH-01,8.0
4,2025-08-20 00:12:00+02:00,CMSA-GAKH-01,15.0


Rows: 88800 Zones: 37 Start: 2025-08-20 00:00:00+02:00 End: 2025-08-24 23:57:00+02:00


## 5) Sensorlocaties → GeoJSON

- We lezen `Objectummer` (sensor-id), `Locatienaam`, `Lat/Long`.
- `Lat/Long` wordt geparsed (komma's → punten).
- We maken per sensor een **klein vierkant** polygon (snel voor een choropleth).

> Later kun je dit vervangen door echte polygonen of clusters per gebied.

In [19]:
loc = pd.read_csv(LOC_CSV)
loc = loc.rename(columns={'Objectummer':'zone_id', 'Locatienaam':'name', 'Lat/Long':'latlon'})

def parse_latlon(s):
    s = str(s).strip()
    # Soms 'lat, lon' met komma's als scheiding en decimaal—we filteren getallen eruit
    parts = [p.strip() for p in s.replace(';', ',').split(',')]
    nums = []
    for p in parts:
        try:
            nums.append(float(p.replace(' ','').replace(',', '.')))
        except:
            pass
    if len(nums) >= 2:
        return nums[0], nums[1]
    return np.nan, np.nan

lat, lon = zip(*[parse_latlon(v) for v in loc['latlon']])
loc['lat'] = lat
loc['lon'] = lon
loc = loc[['zone_id','name','lat','lon']]

# Hou alleen sensoren die ook in de flows zitten
present_ids = set(long_df['zone_id'].unique()) & set(loc['zone_id'].unique())
loc = loc[loc['zone_id'].isin(present_ids)].copy()
long_df = long_df[long_df['zone_id'].isin(present_ids)].copy()

def square_around(lat, lon, dlat=0.0008, dlon=0.0012):
    return [
        [lon-dlon, lat-dlat],
        [lon+dlon, lat-dlat],
        [lon+dlon, lat+dlat],
        [lon-dlon, lat+dlat],
        [lon-dlon, lat-dlat]
    ]

features = []
for _, r in loc.iterrows():
    if np.isnan(r['lat']) or np.isnan(r['lon']):
        continue
    poly = square_around(r['lat'], r['lon'])
    features.append({
        "type": "Feature",
        "properties": {"zone_id": r['zone_id'], "name": r.get('name', r['zone_id'])},
        "geometry": {"type": "Polygon", "coordinates": [poly]}
    })

gj = {"type":"FeatureCollection","features":features}
with open(OUT_GEOJSON, "w", encoding="utf-8") as f:
    json.dump(gj, f)

print(f"GeoJSON sensors: {len(features)} → {OUT_GEOJSON}")

GeoJSON sensors: 36 → output_notebook_chat\zones.geojson


## 6) Exporteer timeseries voor dashboard

In [20]:
# Minimale structuur voor dashboard/notebooks
timeseries = long_df[['timestamp','zone_id','density']].copy()

# (Optioneel: in toekomst kun je echte features joinen)
timeseries['pt_arrivals'] = np.nan
timeseries['temp'] = np.nan
timeseries['wind'] = np.nan
timeseries['special_event'] = 0

timeseries.to_csv(OUT_TIMESERIES, index=False)
print("OK →", OUT_TIMESERIES, "rows:", len(timeseries))

OK → output_notebook_chat\crowd_timeseries.csv rows: 86400


## 7) Exporteer features-bestand (met placeholders)

We maken alvast `crowd_timeseries_features.csv` zodat je later eenvoudig **AIS/KNMI/WBGT** kunt toevoegen.

In [21]:
features = timeseries.rename(columns={
    'temp':'temp_C', 'wind':'wind_mps'  # align met latere naamgeving
}).copy()

# Voeg WBGT & humidity (rh_pct) als lege placeholder-kolommen toe
features['rh_pct'] = np.nan
features['wbgt_C'] = np.nan

# Voeg AIS placeholders toe
features['vessel_count'] = np.nan
features['vessel_speed_ms'] = np.nan

# Herordenen
cols = ['timestamp','zone_id','density','vessel_count','vessel_speed_ms','temp_C','wind_mps','rh_pct','wbgt_C']
features = features[cols].sort_values(['zone_id','timestamp']).reset_index(drop=True)

features.to_csv(OUT_FEATURES, index=False)
print("OK →", OUT_FEATURES, "rows:", len(features))

OK → output_notebook_chat\crowd_timeseries_features.csv rows: 86400


## 8) Samenvatting & wat nu?

- ✅ **Gemaakt:**  
  - `/mnt/data/crowd_timeseries.csv`  
  - `/mnt/data/zones.geojson`  
  - `/mnt/data/crowd_timeseries_features.csv`  

- 🔁 **Volgende stappen (optioneel):**  
  - Voeg **AIS** toe → tel `vessel_count` binnen straal per zone & 3-min-bin; vul `vessel_speed_ms`.  
  - Voeg **KNMI** toe → vul `temp_C`, `wind_mps`, `rh_pct`.  
  - Voeg **WBGT** toe → vul `wbgt_C` vanuit KNMI WBGT feed.  

- 🧭 **Gebruik in je dashboard/notebook:**  
  - `data_path = "/mnt/data/crowd_timeseries_features.csv"`  
  - `geo_path  = "/mnt/data/zones.geojson"`  
  - `step_minutes = 3`